# Import Modules

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=50

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "LeafDisease",
    shuffle= True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

In [ ]:
for images, _ in dataset.take(1):  # Adjust the number of batches as needed
    min_value = tf.reduce_min(images).numpy()
    max_value = tf.reduce_max(images).numpy()
    print("Minimum value:", min_value)
    print("Maximum value:", max_value)

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
len(dataset)

In [ ]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

In [ ]:
plt.figure(figsize=(15, 15))
for image_batch, labels_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[labels_batch[i]])
        plt.axis("off")

In [ ]:
train_size = 0.8
len(dataset)*train_size

In [ ]:
train_ds = dataset.take(899)
len(train_ds)

In [ ]:
test_ds = dataset.skip(899)
len(test_ds)

In [ ]:
val_size=0.1
len(dataset)*val_size

In [ ]:
val_ds = test_ds.take(112)
len(val_ds)

In [ ]:
test_ds = test_ds.skip(112)
len(test_ds)


In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [ ]:
len(train_ds)

In [ ]:
len(val_ds)

In [ ]:
len(test_ds)

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
class Autoencoder(tf.keras.Model):
    def __init__(self):
        super().__init__()  # Correct super() call without arguments

        # N, 1, 28, 28
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Conv2D(16, (3, 3), strides=2, padding="same", activation="relu"),  # -> N, 16, 14, 14
            tf.keras.layers.Conv2D(32, (3, 3), strides=2, padding="same", activation="relu"),  # -> N, 32, 7, 7
            tf.keras.layers.Conv2D(64, (7, 7), activation="relu")  # -> N, 64, 1, 1
        ])

        # N, 64, 1, 1
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Conv2DTranspose(32, (7, 7), activation="relu"),  # -> N, 32, 7, 7
            tf.keras.layers.Conv2DTranspose(16, (3, 3), strides=2, padding="same", output_padding=1, activation="relu"),  # N, 16, 14, 14
            tf.keras.layers.Conv2DTranspose(1, (3, 3), strides=2, padding="same", output_padding=1, activation="sigmoid")  # N, 1, 28, 28
        ])

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
model = Autoencoder()

# Define the loss function (Mean Squared Error)
loss_function = tf.keras.losses.MeanSquaredError()

# Define the optimizer (Adam optimizer)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)

In [ ]:
num_epochs = 5
outputs = []

for epoch in range(num_epochs):
    x = 0
    for img, _ in dataset:
        with tf.GradientTape() as tape:
            recon = model(img)
            loss = tf.keras.losses.MeanSquaredError()(img, recon)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        x = x+1
        print(x)
    print(f'Epoch:{epoch+1}, Loss:{loss:.4f}')
    outputs.append((epoch, img, recon))


In [ ]:
import numpy as np

for k in range(0, num_epochs, 4):
    plt.figure(figsize=(40, 30))
    plt.gray()
    imgs = outputs[k][1].numpy()
    recon = outputs[k][2].numpy()
    for i, item in enumerate(imgs):
        if i >= 3:
            break
        plt.subplot(2, 9, i + 1)
        # item: (height, width, channels)
        plt.imshow(item[:, :, 0], cmap='gray')
        
    for i, item in enumerate(recon):
        if i >= 3:
            break
        plt.subplot(2, 9, 9 + i + 1)  # row_length + i + 1
        # item: (height, width, channels)
        plt.imshow(item[:, :, 0], cmap='gray')



In [ ]:
for k in range(0, num_epochs, 4):
    plt.figure(figsize=(9, 2))
    recon = outputs[k][2].numpy()
    
    for i in range(9):
        plt.subplot(2, 9, i + 1)
        plt.imshow(recon[i, :, :], cmap='gray')
        plt.axis('off')  # Remove axis labels and ticks

plt.show()  # Display the reconstructed images


In [ ]:
import os
import shutil
import random



In [ ]:
def create_test_dataset(train_folder, test_folder, split_ratio=0.2):
    # Create the test dataset folder if it doesn't exist
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)

    # Iterate through each class in the train dataset folder
    for class_folder in os.listdir(train_folder):
        # Skip hidden files (e.g., .DS_Store)
        if class_folder.startswith('.'):
            continue

        class_path = os.path.join(train_folder, class_folder)
        
        # Create the corresponding class folder in the test dataset
        test_class_path = os.path.join(test_folder, class_folder)
        if not os.path.exists(test_class_path):
            os.makedirs(test_class_path)

        # Get a list of all images in the class folder
        images = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

        # Calculate the number of images to move to the test dataset
        num_test_images = int(split_ratio * len(images))

        # Randomly select images for the test dataset
        test_images = random.sample(images, num_test_images)

        # Move selected images to the test dataset folder
        for image in test_images:
            src_path = os.path.join(class_path, image)
            dest_path = os.path.join(test_class_path, image)
            shutil.move(src_path, dest_path)

if __name__ == "__main__":
    # Set your train and test dataset folder paths
    train_dataset_folder = "/Users/aayushrangra/Desktop/GrapeVine_Disease_Detection/LeafDisease"
    test_dataset_folder = "/Users/aayushrangra/Desktop/GrapeVine_Disease_Detection/TestData"

    # Set the split ratio (e.g., 0.2 for 20% test, 80% train)
    split_ratio = 0.2

    # Create the test dataset
    create_test_dataset(train_dataset_folder, test_dataset_folder, split_ratio)


In [ ]:

def check_dataset_length(dataset_folder):
    # Iterate through each class in the dataset folder
    for class_folder in os.listdir(dataset_folder):
        # Skip hidden files (e.g., .DS_Store)
        if class_folder.startswith('.'):
            continue

        class_path = os.path.join(dataset_folder, class_folder)

        # Get a list of all images in the class folder
        images = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

        # Check the length of the images list
        num_images = len(images)

        # Output the number of images for each class
        print(f"Class '{class_folder}' has {num_images} images.")

if __name__ == "__main__":
    # Set your dataset folder path
    Test = "/Users/aayushrangra/Desktop/GrapeVine_Disease_Detection/TestData"

    # Check the length of the dataset
    check_dataset_length(Test)


In [ ]:
import torch
from torch import nn
import torch.optim as optim





In [ ]:
inputs = torch.randn(10, 2)
targets = torch.randn(10, 1)

# forward pass
outputs = model(inputs)

# calculate the loss
criterion = nn.MSELoss()
loss = criterion(outputs, targets)

# backward pass
optimizer.zero_grad()
loss.backward()

# optimization step
optimizer.step()

In [ ]:
num_epochs=5000
for epoch in range(num_epochs):
    # forward pass
    outputs = model(inputs)
    
    # calculate the loss
    loss = criterion(outputs, targets)
    
    # backward pass
    optimizer.zero_grad()
    loss.backward()
    
    # optimization step
    optimizer.step()

    # print the loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# After training, you can use the trained model for predictions
# For example:
test_input = torch.randn(1, 2)
predicted_output = model(test_input)
print("Predicted Output:", predicted_output.item())

In [ ]:
from torch.optim.optimizer import Optimizer
import torch_optimizer as optimizer

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

from diffgrad import DiffGrad

In [7]:
from tensorflow.keras.optimizers import register

ImportError: cannot import name 'register' from 'tensorflow.keras.optimizers' (/Applications/anaconda3/lib/python3.9/site-packages/keras/api/_v2/keras/optimizers/__init__.py)

In [2]:
# Assuming you have your input data and target data as 'inputs' and 'targets'
inputs = tf.random.normal((10, 2))
targets = tf.random.normal((10, 1))

# model definition
model = models.Sequential([
    layers.Dense(3, activation='relu', input_shape=(2,)),
    layers.Dense(1)
])

# optimizer definition
optimizer = DiffGrad()

# compile the model
model.compile(optimizer=optimizer, loss='mean_squared_error')

# number of training epochs
num_epochs = 1000

# training loop
for epoch in range(num_epochs):
    # forward pass
    outputs = model(inputs)
    
    # calculate the loss
    loss = model.evaluate(inputs, targets, verbose=0)
    
    # backward pass
    with tf.GradientTape() as tape:
        current_loss = model(inputs, training=True)
    
    # optimization step
    gradients = tape.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # print the loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}')

# After training, you can use the trained model for predictions
# For example:
test_input = tf.random.normal((1, 2))
predicted_output = model.predict(test_input)
print("Predicted Output:", predicted_output[0, 0])


2023-11-14 03:46:52.236555: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Could not interpret optimizer identifier: <diffgrad.DiffGrad object at 0x7fd719e03430>

In [1]:
import os
import shutil
import random

def create_validation_set(data_dir, validation_dir, validation_percent=0.1):
    # Create the validation set directory if it doesn't exist
    if not os.path.exists(validation_dir):
        os.makedirs(validation_dir)

    # Iterate through each class folder in the training set
    for class_folder in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_folder)
        validation_class_path = os.path.join(validation_dir, class_folder)

        # Create the class folder in the validation set
        if not os.path.exists(validation_class_path):
            os.makedirs(validation_class_path)

        # List all files in the class folder
        files = os.listdir(class_path)

        # Calculate the number of files to move to the validation set
        num_validation_files = int(validation_percent * len(files))

        # Randomly select files for validation
        validation_files = random.sample(files, num_validation_files)

        # Move selected files to the validation set folder
        for file in validation_files:
            src_path = os.path.join(class_path, file)
            dest_path = os.path.join(validation_class_path, file)
            shutil.move(src_path, dest_path)

# Example usage
create_validation_set('/Users/aayushrangra/Desktop/GrapeVine_Disease_Detection/custom_dataset/train', '/Users/aayushrangra/Desktop/GrapeVine_Disease_Detection/custom_dataset', validation_percent=0.1)


NotADirectoryError: [Errno 20] Not a directory: '/Users/aayushrangra/Desktop/GrapeVine_Disease_Detection/custom_dataset/train/.DS_Store'